- 최종 데이터셋을 활용한 비율 계산 및 시각화

In [ ]:
import pandas as pd

file_path = "./custom_data/merge/order_customer_merged.csv"

# 병합된 주문 데이터
order_df = pd.read_csv(file_path)

In [ ]:
# 각 도시와 주에 있는 고객 수를 집계
city_state_counts = (
    order_df.groupby(["customer_state", "customer_city"])
    .size()
    .reset_index(name="customer_count")
)

city_state_counts.sort_values("customer_count", ascending=False)

In [ ]:
# 각 도시별 고객 수를 계산
city_counts = (
    order_df.groupby(["customer_state", "customer_city"])
    .size()
    .reset_index(name="customer_count")
)

# 각 주별 고객 수를 계산
state_counts = (
    order_df.groupby("customer_state").size().reset_index(name="state_customer_count")
)

In [ ]:
# 정보 병합
merged_df = pd.merge(city_counts, state_counts, on="customer_state")

# 'state_customer_count' 내림차순, 같은 state 내에서는 'customer_count' 내림차순으로 정렬
sorted_merged_df = merged_df.sort_values(
    by=["state_customer_count", "customer_count"], ascending=[False, False]
)

In [ ]:
# 해안, 내륙지역 분류
coastal_s = [
    "AP", "PA", "MA", "PI", "CE", "RN", "PB", "PE", "AL", "SE", "BA", "ES", "RJ", "SP", "PR", "SC", "RS"
]
inland_s = [
    "AM", "RR", "AC", "RO", "MT", "GO", "MG", "MS", "TO", "DF"
]
state_mapping = {
    "AC": "Acre", "AL": "Alagoas", "AP": "Amapá", "AM": "Amazonas", "BA": "Bahia", 
    "CE": "Ceará", "DF": "Distrito Federal", "ES": "Espírito Santo", "GO": "Goiás", 
    "MA": "Maranhão", "MT": "Mato Grosso", "MS": "Mato Grosso do Sul", "MG": "Minas Gerais", 
    "PA": "Pará", "PB": "Paraíba", "PR": "Paraná", "PE": "Pernambuco", "PI": "Piauí", 
    "RN": "Rio Grande do Norte", "RJ": "Rio de Janeiro", "RS": "Rio Grande do Sul", 
    "RO": "Rondônia", "RR": "Roraima", "SC": "Santa Catarina", "SP": "São Paulo", 
    "SE": "Sergipe", "TO": "Tocantins"
}

coastal = [state_mapping[code] for code in coastal_s]
inland = [state_mapping[code] for code in inland_s]

In [ ]:
# 해안/내륙 컬럼에 데이터 추가
sorted_merged_df["region"] = sorted_merged_df["customer_state"].apply(
    lambda x: "coastal" if x in coastal else "inland"
)

In [ ]:
file_path = "./custom_data/final/state_sports_df.csv"

# 스포츠 관련 용품 구매 데이터
sports_df = pd.read_csv(file_path)

In [ ]:
# 절대치가 아닌 비율로 계산
sports_df["purchase_rate"] = (
    sports_df["total"] / sorted_merged_df["state_customer_count"]
)

In [ ]:
# 비율로 계산한 데이터셋 새로운 csv 파일로 저장
sports_df.to_csv("sports_proportion_df.csv", index=False)

In [ ]:
# 해안/내륙별로 비율 합계
proportion_region = (
    sports_df.groupby(["region"])["purchase_rate"]
    .sum()
    .reset_index(name="proportion_region")
)

In [ ]:
# 비율 시각화
import seaborn as sns
import matplotlib.pyplot as plt

sns.barplot(
    proportion_region,
    x="region",
    y="proportion_region",
    hue="region",
    legend=False,
    palette="viridis",
)

plt.title("purchase rate between coastal and inland")
plt.show()

In [ ]:
# 파이 차트로 비율 시각화
plt.figure(figsize=(8, 8))

colors = ["cornflowerblue", "yellowgreen"]

explode = (0.1, 0)

plt.pie(
    proportion_region["proportion_region"],
    labels=proportion_region["region"].unique(),
    autopct="%1.1f%%",
    colors=colors,
    explode=explode,
    shadow=True,
)

plt.title("purchase rate of sports products between coastal and inland")
plt.show()